### Importing Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from requests import get
import time
from selenium.webdriver.common.by import By
from getpass import getpass

### Navigate to the webiste using Selenium

In [2]:
driver = webdriver.Chrome(r"C:\Users\Cellfixx\Downloads\chromedriver_win32\chromedriver.exe")
driver.get("https://accounts.craigslist.org/login/home")
driver.maximize_window()
pwd = getpass('Enter Password:')

username_box = driver.find_element_by_id("inputEmailHandle")
username_box.send_keys("cpsclangara@gmail.com")

password_box = driver.find_element_by_id("inputPassword")
password_box.send_keys(pwd)

#login_btn.click()
driver.find_element(By.ID, "login").click()
#navigate to real estate listings for sale
driver.find_element_by_link_text("craigslist").click()
driver.find_element_by_class_name("rea").click()

Enter Password:········


### Scraping Real estate data from Craiglist

In [4]:
#%%time
#counter to check the number of house ads scraped
house_no =0

#dicitionary to store housing details
house_details = {}

#searching criteria
regex = re.compile('[ft2]+')
furn_search = re.compile('furnished')
type_search = re.compile('apartment|condo|duplex|flat|house|townhouse|land')
parking_search = re.compile('attached garage|detached garage|street parking|no parking')

#original url
url = 'https://vancouver.craigslist.org/d/real-estate/search/rea'

#looping through each main page
while True:
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    houses = soup.find_all('p', class_='result-info')
    #looping through each ad in a page
    for house in houses:
        link = house.find('a', class_='result-title hdrlnk').get('href')
        #print(link)
        house_response = requests.get(link)
        house_data = house_response.text
        house_soup = BeautifulSoup(house_data, 'html.parser')
        
        #Getting the price
        price_tag = house_soup.find('span', class_='price')
        price = price_tag.text[1:] if price_tag else 'NA'
        
        #Getting area and bedroom
        housing_tag = house_soup.find('span', class_='housing')

        if housing_tag:
            bedroom = housing_tag.text[2:3]
            #print(len(housing_tag.text))
            #print(housing_tag.text)

            c = housing_tag.text.split('-')
            for i in c:
                i = i.strip()
                #print(i)
                ft = re.compile('[ft2]+')
                start_post = re.compile('\s')
                if ft.search(i):
                        #print(i)
                        end= regex.search(i).span()[0]
                        #print(regex.search(i).span())
                        #print(end)
                        if start_post.search(i):
                            start = start_post.search(i).span()[1]
                            #print(start_post.search(i).span())
                        else:
                            start =0
                        #print(start)
                        #print(i[start:end])
                        area = i[start:end]

            if area is None:
                area = 'NA'
        else:
            area = 'NA'
            bedroom ='NA'

        #Getting title of the ad
        title_tag = house_soup.find('span', id='titletextonly')
        title = title_tag.text if title_tag else 'NA'
        #print(title)
        
        #Getting the neighborhood
        place_tag = house_soup.find('small')
        place = place_tag.text[2:-1] if place_tag else 'NA'

        #Getting the posted date time
        post_datetime_tag = house_soup.find('time', class_='date timeago')
        post_datetime = post_datetime_tag['datetime'] if post_datetime_tag else 'NA'

        divs = house_soup.find('div',class_='mapAndAttrs')
        #print(divs.text)
        #checking if the estate is furnished or unfurnished

        if (furn_search.search(divs.text) is not None):
            finishing = 'furnished'
        else:
            finishing = 'unfurnished'

        #checking the housing type
        if (type_search.search(divs.text) is not None):
            house_type = type_search.search(divs.text).group()
 
        else:
            house_type = 'NA'

        #checking the parking type
        if (parking_search.search(divs.text) is not None):
            parking_type = parking_search.search(divs.text).group()
        else:
            parking_type = 'NA'

        #checking number of bedroom and bathroom
        data = divs.findNext('p').findNext('span')
        #print(data)
        if ((data.text is not None) and (data.text.find('/')>0)):
            # if there is no value in the bedroom, then check this value
            if bedroom is None:
                bd = data.text.split('/')[0].strip() 
                #print(bd)
                if (bd.find('BR')>0):
                    bedroom = bd[:-2]
                else:
                    bedroom = 'NA'
            #setting the number of bathroom value
            if(data.text.split('/')[1].strip()):
                bth = data.text.split('/')[1].strip()
                #print(bth)
                if (bth.find('Ba')>0):
                    bathroom = bth[:-2]
                else:
                    bathroom = 'NA'

        house_no+=1
    
        #Making an entry for each ad in a dictionary
        house_details[house_no] = [price, bedroom, area, title, place, post_datetime, finishing, house_type, parking_type,
                                   bathroom, link]
     
    #Getting the next page link in the original link
    url_tag = soup.find('a', class_='button next')
    if url_tag.get('href'):
        url = 'https://vancouver.craigslist.org/d/real-estate/search/rea' + url_tag.get('href')
        #print(url)
    else:
        # breaking from the loop in case there is no more link .ie in the last page
        break
    
    #putting a counter to fetch only 1200
    if(house_no==240):
        break

#Building a data frame from the dictionary created, updated in the above loop
house_df = pd.DataFrame.from_dict(house_details,orient='index', columns=['Price', 'Bedroom', 'Area', 'Title', 'Place', 'Post Timing',
                                                                        'Finishing','House Type', 'Parking Type','Bathroom','Link'])

### Printing the scrapped Data

In [5]:
house_df

,Price,Bedroom,Area,Title,Place,Post Timing,Finishing,House Type,Parking Type,Bathroom,Link
1,648000,1,771,Brand New 1 Bed & Den in Award Winning Compass...,402 680 Seylynn Crescent,2020-07-28T13:43:24-0700,unfurnished,condo,attached garage,1,https://vancouver.craigslist.org/van/reb/d/nor...
2,179000,1,600,investment property 4-6% ROI better than the bank,Courtenay- Vancouver Island,2020-07-15T09:49:43-0700,unfurnished,apartment,street parking,1,https://vancouver.craigslist.org/van/reo/d/cou...
3,1099000,2,11,2 Storey Condo in Beautiful Compass in Seylynn,610 680 Seylynn Crescent,2020-07-28T13:43:10-0700,unfurnished,condo,attached garage,2,https://vancouver.craigslist.org/van/reb/d/nor...
4,2895000,NA,NA,"4.67 ACRES and 12,000+ SQ/FT ESTATE HOME.",NA,2020-07-28T13:52:10-0700,unfurnished,land,NA,2,https://vancouver.craigslist.org/rds/reb/d/lan...
5,2976,6,691,★ ★TURNKEY RETAIL SPACE IN BURNABY FOR LEASE ★ ★,Burnaby,2020-07-13T14:51:38-0700,unfurnished,apartment,NA,2,https://vancouver.craigslist.org/bnc/reb/d/bur...
...,...,...,...,...,...,...,...,...,...,...,...
236,1368000,3,,Totally Renovated Townhome!,Vancouver,2020-07-27T08:45:44-0700,unfurnished,townhouse,NA,3,https://vancouver.craigslist.org/van/reb/d/van...
237,419000,1,715,Great Space!,Langley,2020-07-27T08:54:16-0700,unfurnished,apartment,NA,1,https://vancouver.craigslist.org/rds/reb/d/lan...
238,1198000,3,,Beautiful 3 Bed 2 Bath Home In Prime Burnaby E...,Burnaby,2020-07-08T08:25:42-0700,unfurnished,house,NA,2,https://vancouver.craigslist.org/bnc/reb/d/bur...
239,459900,3,,Beautiful 3 Bed 2.5 Bath Townhome in Cedarbroo...,Mission,2020-07-22T08:36:55-0700,unfurnished,apartment,attached garage,2.5,https://vancouver.craigslist.org/rds/reb/d/mis...


### Saving data into CSV file

In [6]:
#Saving the dataframe into csv file
house_df.to_csv("Craiglist_HouseData.csv")